# MPIIGaze preprocessing (RU)

Этот ноутбук показывает, как мы обрабатываем исходники `MPIIGaze` в проекте:

- читаем `Annotation Subset`
- находим исходные кадры в `Data/Original`
- строим кропы `face / left_eye / right_eye`
- считаем нормированные координаты зрачка
- сохраняем `metadata.csv`.


## 1. Где код preprocessing

- `gaze_tracker/datasets/mpiigaze_preprocess.py`
- CLI: `python -m gaze_tracker.datasets.mpiigaze_preprocess`

Подробное текстовое описание:
- `docs/dataset_mpiigaze_preprocessing.md`


In [ ]:
# Проверка, что исходники датасета на месте
from pathlib import Path

root = Path("MPIIGaze")
print("MPIIGaze exists:", root.exists())
for p in [
    root / "Annotation Subset",
    root / "Data" / "Original",
    root / "Evaluation Subset",
]:
    print(p, "->", p.exists())


## 2. Формат строки аннотации

Пример строки в `Annotation Subset/pXX.txt`:

```text
day13/0203.jpg 614 355 684 357 769 368 842 379 643 524 768 535 640 356 803 375
```

Интерпретация:
1. путь к кадру
2. 2 точки левого глаза (уголки)
3. 2 точки правого глаза (уголки)
4. 2 точки рта
5. левый и правый центр зрачка


In [ ]:
# Покажем первые 3 строки p00.txt
from itertools import islice

ann_path = Path("MPIIGaze") / "Annotation Subset" / "p00.txt"
with ann_path.open("r", encoding="utf-8") as f:
    for line in islice(f, 3):
        print(line.strip())


## 3. Smoke-test preprocessing (200 сэмплов)

Если запускаешь ячейку в Jupyter, команда может занять немного времени.


In [ ]:
# Вариант через Python API (удобно для ноутбука)
from gaze_tracker.datasets.mpiigaze_preprocess import preprocess_mpiigaze

stats = preprocess_mpiigaze(
    mpiigaze_root=Path("MPIIGaze"),
    output_dir=Path("data/mpiigaze_processed_sample"),
    max_samples=200,
    save_images=True,
)
stats


In [ ]:
# Эквивалентная команда CLI
print(
    "python -m gaze_tracker.datasets.mpiigaze_preprocess "
    "--mpiigaze-root MPIIGaze "
    "--output-dir data/mpiigaze_processed_sample "
    "--max-samples 200"
)


## 4. Проверка результата

После выполнения должны появиться:

- `data/mpiigaze_processed_sample/metadata.csv`
- `data/mpiigaze_processed_sample/images/face/*.jpg`
- `data/mpiigaze_processed_sample/images/left_eye/*.jpg`
- `data/mpiigaze_processed_sample/images/right_eye/*.jpg`


In [ ]:
import csv

meta_path = Path("data/mpiigaze_processed_sample/metadata.csv")
print("metadata exists:", meta_path.exists())

if meta_path.exists():
    with meta_path.open("r", encoding="utf-8", newline="") as f:
        reader = csv.DictReader(f)
        rows = []
        for i, row in enumerate(reader):
            if i >= 5:
                break
            rows.append(row)
    rows


In [ ]:
# Визуальная проверка одного примера
import cv2
from matplotlib import pyplot as plt

sample_face = Path("data/mpiigaze_processed_sample/images/face")
sample_left = Path("data/mpiigaze_processed_sample/images/left_eye")
sample_right = Path("data/mpiigaze_processed_sample/images/right_eye")

if sample_face.exists():
    face_img_path = sorted(sample_face.glob("*.jpg"))[0]
    left_img_path = sorted(sample_left.glob("*.jpg"))[0]
    right_img_path = sorted(sample_right.glob("*.jpg"))[0]

    face = cv2.cvtColor(cv2.imread(str(face_img_path)), cv2.COLOR_BGR2RGB)
    left = cv2.cvtColor(cv2.imread(str(left_img_path)), cv2.COLOR_BGR2RGB)
    right = cv2.cvtColor(cv2.imread(str(right_img_path)), cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1); plt.imshow(face); plt.title("face"); plt.axis("off")
    plt.subplot(1, 3, 2); plt.imshow(left); plt.title("left_eye"); plt.axis("off")
    plt.subplot(1, 3, 3); plt.imshow(right); plt.title("right_eye"); plt.axis("off")
    plt.tight_layout()
else:
    print("Сначала запусти preprocessing (предыдущие ячейки).")


## 5. Полная обработка

Когда smoke-test отработал корректно, запускаем полный прогон:

```powershell
python -m gaze_tracker.datasets.mpiigaze_preprocess `
  --mpiigaze-root MPIIGaze `
  --output-dir data/mpiigaze_processed
```

Опционально можно собрать только `metadata.csv` без кропов:

```powershell
python -m gaze_tracker.datasets.mpiigaze_preprocess `
  --mpiigaze-root MPIIGaze `
  --output-dir data/mpiigaze_processed_meta_only `
  --no-images
```
